# Лабораторная работа №4 Генерация текстов на основе LSTM

## Импорт библиотек

In [ ]:
import re

import numpy as np

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchtext.vocab import build_vocab_from_iterator

from sklearn.model_selection import train_test_split


from tqdm.auto import tqdm

## Device

In [ ]:
# Check that MPS is available
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")

else:
    mps_device = torch.device("mps")
    print(mps_device)

## Функция обучение 

In [ ]:
def train(model: nn.Module, crterion, optimizer , n_epochs, train_loader):
    model.train()
    for epoch in range(n_epochs):
        for batch in tqdm(train_loader , desc=f'Training epoch {epoch + 1}:'):
            inputs = batch['input']
            labels = batch['label']
            print(inputs.dtype, inputs.shape)
            output = model(inputs)


    model.eval()
            

## Задание 1. Загрузите текст из произведений Ницше 
('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt').


Выведете следующее:
- А) длину всего корпуса;
- Б) количество предложений;
- В) сколько всего символов используется?


In [ ]:
with open('nietzsche.txt', 'r') as file:
    lines = file.read()

text_nietzsche = lines
text_nietzsche[:100]

In [ ]:
text_nietzsche = text_nietzsche.lower()
text_nietzsche = re.sub(r'\s', ' ', text_nietzsche) # replace \n \t and etc.
text_nietzsche = re.sub(r'\s{2,}', ' ', text_nietzsche) # replace repeated whitespace

### Длина всего корпуса

In [ ]:
len(text_nietzsche)

### Количество предложений

In [ ]:
len(sent_tokenize(text_nietzsche))

### Cколько всего символов используется?

In [ ]:
len(set(text_nietzsche))

## Задание 2. Сократите текст наполовину избыточными последовательностями символов maxlen

In [ ]:
vocab = build_vocab_from_iterator(text_nietzsche, specials=["<unk>"])
token2inx = vocab.get_stoi()
inx2token = {inx: token for token, inx in vocab.get_stoi().items()}

In [ ]:
token2inx

In [ ]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text_nietzsche) - maxlen, step):
    sentences.append(text_nietzsche[i: i + maxlen])
    next_chars.append(text_nietzsche[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(token2inx)))
y = np.zeros((len(sentences), len(token2inx)))
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, token2inx[char]] = 1
    y[i, token2inx[next_chars[i]]] = 1

In [ ]:
x.shape

In [ ]:
sentences[0]

In [ ]:
x[0][0]

У нас в датасете 199607 40 символьных отрезков и ответ следущая буква. В каждом sample 40 списков one-hot, где одна единица обозначает букву.

## Задание 3. Создайте модель LSTM для генерации текста. 

- А) Напишите вспомогательную функцию для выборки индекса из массива вероятностей
- Б) Напишите функцию, которая будет вызываться в конце каждой эпохи и печатать сгенерированный текст
- В) Запустите модель на обучение Имейте ввиду, что требуется не менее 20 эпох, прежде чем сгенерированный текст начнет звучать связно. Рекомендуется запускать этот скрипт на графическом процессоре, так как рекуррентные сети требуют довольно больших вычислительных затрат.
- Г) Проверьте работу модели в онлайн режиме.

### Train and test

In [ ]:
X_train, X_test , y_train, y_test = train_test_split(x,y, test_size = 0.2)

In [ ]:
len(X_train) , len(X_test) , len(y_train) , len(y_test)

### Dataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input = torch.LongTensor(self.data[idx]).to(mps_device)
        label = torch.LongTensor(self.labels[idx]).to(mps_device)
        sample = {'input': input, 'label': label}
        return sample

In [ ]:
train_dataset= CustomDataset(X_train, y_train)

test_dataset= CustomDataset(X_test, y_test)

### DataLoader

In [ ]:
train_dataloader = DataLoader(train_dataset,  batch_size=256)

test_dataloader = DataLoader(test_dataset, batch_size=256)

### Функция выборки индекса из массива вероятностей

### Функция для генерации текста

In [ ]:
def generate_text():
    pass

### Модель 

In [ ]:
class AnpilovGpt(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size1, hidden_size2 ):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings = vocab_size, embedding_dim = embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size1, batch_first=True) # (batch_size, seq_len, input_size)
        self.linear = nn.Linear(in_features = hidden_size1 , out_features = hidden_size2 )
        self.projection = nn.Linear(in_features = hidden_size2 , out_features = vocab_size) # vocab_size = num_classes

        self.dropout = nn.Dropout(p = 0.1) # defult p = 0.5
        self.tanh = nn.Tanh() # against vanish gradient

    def forward(self, input):
        x = self.embedding(input).view(256, 40, -1)
        print(x.shape)
        x, _ = self.lstm(x)
        x = self.linear(x)
        x = self.projection(x)
        return x
        
    

### Проверка модели на свой текст

### Обучение 

In [ ]:
n_epochs = 20
vocab_size = len(vocab) # 57
embedding_dim = 200
hidden_size1 = 256
hidden_size2 = 256


model = AnpilovGpt(vocab_size, embedding_dim, hidden_size1, hidden_size2).to(mps_device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters()) # lr = 0.001

train(model, criterion, optimizer, n_epochs, train_dataloader)

## Задание 4. Создайте самостоятельно генерацию текста для РУССКОЯЗЫЧНОГО НАБОРА глав Wikibooks.
Полный текст Wikibooks содержит более 270000 глав на 12 языках https://www.kaggle.com/datasets/dhruvildave/wikibooks-dataset/data
